<a href="https://colab.research.google.com/github/raqueeb/nlp_bangla/blob/master/Chap_7_transformers_bert_Bangla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## হাতেকলমে ন্যাচারাল ল্যাঙ্গুয়েজ প্রসেসিং: শুরুর ধারণা

[৭ম অধ্যায়](https://aiwithr.github.io/nlpbook/)

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [3]:
!ls ~/.kaggle

kaggle.json


In [4]:
from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()

In [5]:
api.dataset_list_files('furcifer/bangla-newspaper-dataset').files

[data/data.json, data_v2/data_v2.json]

In [6]:
api.dataset_download_file('furcifer/bangla-newspaper-dataset','data_v2/data_v2.json')

False

In [7]:
!unzip data_v2.json.zip

Archive:  data_v2.json.zip
replace data_v2.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [8]:
import json
with open('data_v2.json', encoding='utf-8') as f:
    datastore = json.load(f)

sentences = []
labels = []

for item in datastore:
    sentences.append(item['content'])
    labels.append(item['category'])

In [9]:
set(labels)

{'bangladesh',
 'economy',
 'education',
 'entertainment',
 'international',
 'life-style',
 'opinion',
 'sports',
 'technology'}

In [10]:
categories = list(set(labels))

In [11]:
categories

['life-style',
 'technology',
 'international',
 'entertainment',
 'sports',
 'education',
 'economy',
 'bangladesh',
 'opinion']

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
  sentences, labels, test_size=0.2, random_state=42)

In [13]:
!pip install ktrain

In [14]:
import ktrain
from ktrain import text

In [15]:
MODEL_NAME = 'sagorsarker/bangla-bert-base'
t = text.Transformer(MODEL_NAME, maxlen=500, class_names = categories)
trn = t.preprocess_train(X_train, y_train)
val = t.preprocess_test(X_test, y_test)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

preprocessing train...
language: bn
train sequence lengths:
	mean : 268
	95percentile : 669
	99percentile : 1079


/usr/local/lib/python3.6/dist-packages/ktrain/utils.py:589: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  if self.get_classes(): warnings.warn('class_names argument was ignored, as they were extracted from string labels in dataset')


Is Multi-Label? False
preprocessing test...
language: bn
test sequence lengths:
	mean : 269
	95percentile : 669
	99percentile : 1075


In [ ]:
%matplotlib inline
learner.lr_find(show_plot=True, max_epochs=1)

simulating training for different learning rates... this may take a few moments...
 4813/54462 [=>............................] - ETA: 11:00:04 - loss: 1.2342 - accuracy: 0.6309

In [ ]:
learner.fit_onecycle(8e-5, 2)

In [ ]:
learner.validate(class_names=t.get_classes())

In [ ]:
# the one we got most wrong
learner.view_top_losses(n=1, preproc=t)

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

In [ ]:
predictor.predict('বাংলাদেশে নদ-নদীগুলো ছড়িয়ে আছে জালের মতো। ঐতিহাসিকভাবেই এই অঞ্চলের নদ-নদীর গতি–প্রকৃতি ও প্রবাহ জটিল।')

In [ ]:
predictor.explain('শিক্ষা মন্ত্রণালয়ে এমন প্রস্তাব দিয়েছে মাউশি। মাউশির কর্মকর্তারা বলছেন, মন্ত্রণালয় অনুমোদন দিলেই সবাইকে সময়সহ অন্যান্য আনুষঙ্গিক বিষয় জানিয়ে দেওয়া হবে। তখন বেসরকারি বিদ্যালয়গুলোও কীভাবে নিজস্ব ব্যবস্থায় আবেদনপত্র বিতরণ ও লটারির কাজটি করবে, তা-ও জানিয়ে দেওয়া হবে। একই সঙ্গে ঢাকার বাইরের সরকারি বিদ্যালয়গুলোর বিষয়েও দিকনির্দেশনা দেওয়া হবে।')

In [ ]:
predictor.predict_proba('বঙ্গবন্ধু টি–টোয়েন্টি কাপে কাল জেমকন খুলনার হয়ে খেলার ম্যাচে ৪ ওভারে ২৮ রানে ১ উইকেট নেন মাশরাফি বিন মুর্তজা।')

In [ ]:
predictor.get_classes()